In [2]:
# using pylab magic function for convenience
%pylab inline
# pylab to import matplotlib and pandas etc. 
# inline lets you plot figures inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [6]:
def fetch(url):
    import requests, gzip, os, hashlib, numpy
    fp = os.path.join("/tmp", hashlib.md5(url.encode('utf-8')).hexdigest())
    if not os.path.isfile(fp):
        # open in read binary mode
        with open(fp, "rb") as f:
            dat = f.read()
    else :
        # open in write binary mode
        with open(fp, "wb") as f:
            dat = requests.get(url).content
            f.write(dat)
    return numpy.frombuffer(gzip.decompress(dat), dtype = np.uint8)